In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import numpy as np
import gzip
import pickle
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gc
from torch.cuda.amp import autocast, GradScaler

import sys
sys.path.append('..')
# from slp_package.slp_functions import create_merged_game_data_df
from slp_package.input_dataset import InputDataSet
import slp_package.pytorch_functions as slp_pytorch_functions


In [14]:
source_data = ['ranked']

general_features = {
    # 'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
    'num_players': [2],
    'conclusive': [True]
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'MARIO', 'PEACH', 'ROY', 'LUIGI', 'YOUNG_LINK', 'DONKEY_KONG', 'PICHU', 'KIRBY'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'PEACH', 'LUIGI', 'DONKEY_KONG'],
    # 'type_name': ['HUMAN']
    
}
opposing_player_features = {
    # 'character_name': ['MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0']
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['player'], # Can be 'general', 'player
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}

In [15]:
dataset = InputDataSet(source_data, general_features, player_features, opposing_player_features, label_info)

print(dataset.dataset['labels'].value_counts())

FOX               53239
FALCO             45939
MARTH             25406
SHEIK             18083
CAPTAIN_FALCON    17165
Name: labels, dtype: int64


/workspace/slp_jaspar/autoencoder/../slp_package/input_dataset.py:95: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  processed_df = pd.concat([player_1_df, player_2_df], ignore_index=True)


In [16]:
labels_order =  dataset.number_of_segments_per_game(12,6000)
print(labels_order)
labels_order = labels_order['Label'].values


            Label  Count  Shift
0             FOX  52690  51317
1           FALCO  45495  44577
2           MARTH  25195  27200
3           SHEIK  17988  21331
4  CAPTAIN_FALCON  16980  15881


In [17]:
train_df, test_df  = dataset.train_test_split_dataframes(test_ratio = .20, val = False)

In [1]:
train_df.head()

NameError: name 'train_df' is not defined

In [19]:
loaders = slp_pytorch_functions.prepare_data_loaders_no_val(train_df, test_df, 32, 16)

In [20]:
class SimpleCNN(nn.Module):
    """
    A simple Convolutional Neural Network model for processing sequential data.
    """
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.LazyConv1d(32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(2, 2)
        self.fc1 = nn.LazyLinear(128)  # LazyLinear allows deferring the determination of in_features
        self.fc2 = nn.LazyLinear(128, 5)  # Assuming 5 classes for classification

    def forward(self, x):
        """Defines the forward pass of the model."""
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)  # Flatten for the fully connected layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [21]:
model = SimpleCNN().to('cuda')
criterion = nn.CrossEntropyLoss(reduction = 'sum')
optimizer = Adam(model.parameters(), lr=0.001)
# gc.collect()
# torch.cuda.empty_cache()
slp_pytorch_functions.train_model(model, criterion, optimizer, loaders, 'cuda', 20)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
Epoch 1/20:   0%|          | 0/750 [00:00<?, ?batch/s]

ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 

In [12]:
slp_pytorch_functions.evaluate_model(model, loaders['test'], 'cuda')

Test Accuracy: 34.0%


In [ ]:
class CNNAutoencoder(nn.Module):
    def __init__(self):
        super(CNNAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=1),  # example for single-channel input
            nn.ReLU(),
            nn.Conv1d(16, 32, 3, 2, 1),
            nn.ReLU(),
            nn.Conv1d(32, 64, 3, 2, 1),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(64, 32, 3, 2, 1, 1),
            nn.ReLU(),
            nn.ConvTranspose1d(32, 16, 3, 2, 1, 1),
            nn.ReLU(),
            nn.ConvTranspose1d(16, 1, 3, 2, 1, 1),
            nn.Sigmoid()  # use sigmoid for normalization
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x